In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Input, ConvLSTM3D, Conv3D, Flatten, Dense, Dropout, BatchNormalization, GlobalAveragePooling3D, Concatenate, TimeDistributed, MaxPooling3D, AveragePooling3D, GlobalMaxPooling3D, LSTM, Lambda
from tensorflow.keras.models import Model
import numpy as np
import pickle as pkl

2025-03-07 21:56:20.005430: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-07 21:56:20.024343: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741364780.046725 4136096 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741364780.053569 4136096 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-07 21:56:20.078534: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
devices = tf.config.list_physical_devices()
print(devices)

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
tf.config.set_visible_devices([], 'GPU')

In [4]:
# importing data
def returning_pkl_file_data(path : str):
    with open(path, 'rb') as f:
        temp = pkl.load(f)
    return temp

mci_func = returning_pkl_file_data(r"feature_extraction/MCI_func_52_79_95_79_197.pkl")
mci_struct = returning_pkl_file_data(r'feature_extraction/MCI_struct_cat_52_169_205_169.pkl')
cn_func = returning_pkl_file_data(r'feature_extraction/CN_func_42_79_95_79_197.pkl')
cn_struct = returning_pkl_file_data(r'feature_extraction/CN_struct_cat_42_169_205_169.pkl')

In [5]:
func_data = np.concat((mci_func, cn_func), axis=0)
func_data.shape

(94, 79, 95, 79, 197)

In [6]:
struct_data = np.concat((mci_struct, cn_struct), axis=0)
struct_data.shape

(94, 169, 205, 169)

In [7]:
all_labels = np.concat((np.zeros((len(mci_func),)), np.ones((len(cn_func),))))
all_labels.shape

(94,)

In [8]:
func_data = np.expand_dims(func_data, axis=len(func_data.shape))
struct_data = np.expand_dims(struct_data, axis=len(struct_data.shape))

func_data.shape, struct_data.shape

((94, 79, 95, 79, 197, 1), (94, 169, 205, 169, 1))

In [9]:
from sklearn.model_selection import train_test_split

func_train, func_test, struct_train, struct_test, y_train, y_test = train_test_split(
    func_data, struct_data, all_labels, test_size=0.2, random_state=42
)

func_train.shape, func_test.shape, struct_train.shape, struct_test.shape, y_train.shape, y_test.shape

((75, 79, 95, 79, 197, 1),
 (19, 79, 95, 79, 197, 1),
 (75, 169, 205, 169, 1),
 (19, 169, 205, 169, 1),
 (75,),
 (19,))

In [10]:
# --- fMRI Model (ConvLSTM) ---
def build_fmri_model():
    fmri_input = Input(shape=(79, 95, 79, 197, 1), name="fmri_input")
    x = Lambda(lambda x: tf.transpose(x, perm=[0, 4, 1, 2, 3, 5]))(fmri_input)
    # Apply Conv3D independently to each time step
    x = TimeDistributed(Conv3D(32, (3,3,3), padding="same", activation="relu"))(fmri_input)
    x = TimeDistributed(MaxPooling3D(2))(x)  # Downsample spatial dimensions
    x = TimeDistributed(Conv3D(64, (3,3,3), padding="same", activation="relu"))(x)
    x = TimeDistributed(GlobalAveragePooling3D())(x)  # Shape: (batch, time=197, 64)
    
    # Temporal modeling with LSTM
    x = LSTM(128)(x)  # Output shape: (batch, 128)
    return Model(inputs=fmri_input, outputs=x, name="fMRI_Model")

# --- sMRI Model (3D CNN) ---
def build_smri_model():
    smri_input = Input(shape=(169, 205, 169, 1), name="smri_input")

    y = Conv3D(filters=32, kernel_size=(3,3,3), activation="relu", padding="valid")(smri_input)
    y = BatchNormalization()(y)
    y = Conv3D(filters=32, kernel_size=(3,3,3), activation="relu", padding="valid")(y)
    y = BatchNormalization()(y)

    y = GlobalAveragePooling3D()(y)
    y = Dense(128, activation="relu")(y)

    return Model(inputs=smri_input, outputs=y, name="sMRI_Model")

# --- Combine fMRI & sMRI Models ---
def build_combined_model():
    fmri_model = build_fmri_model()
    smri_model = build_smri_model()

    combined = Concatenate()([fmri_model.output, smri_model.output])
    combined = Dense(128, activation="relu")(combined)
    combined = Dropout(0.5)(combined)
    output = Dense(1, activation="sigmoid")(combined)

    model = Model(inputs=[fmri_model.input, smri_model.input], outputs=output, name="Combined_Model")
    return model

# Build & Compile Model
model = build_combined_model()
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
model.summary()


Model: "Combined_Model"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ smri_input          │ (None, 169, 205,  │          0 │ -                 │
│ (InputLayer)        │ 169, 1)           │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ fmri_input          │ (None, 79, 95,    │          0 │ -                 │
│ (InputLayer)        │ 79, 197, 1)       │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3d_2 (Conv3D)   │ (None, 167, 203,  │        896 │ smri_input[0][0]  │
│                     │ 167, 32)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed    │ (None, 79, 95,    │        896 │ fmri_input[0][0]  │
│ (TimeDistributed)   │ 79, 197, 32)      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 167, 203,  │        128 │ conv3d_2[0][0]    │
│ (BatchNormalizatio… │ 167, 32)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed_1  │ (None, 79, 47,    │          0 │ time_distributed… │
│ (TimeDistributed)   │ 39, 98, 32)       │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv3d_3 (Conv3D)   │ (None, 165, 201,  │     27,680 │ batch_normalizat… │
│                     │ 165, 32)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed_2  │ (None, 79, 47,    │     55,360 │ time_distributed… │
│ (TimeDistributed)   │ 39, 98, 64)       │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 165, 201,  │        128 │ conv3d_3[0][0]    │
│ (BatchNormalizatio… │ 165, 32)          │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed_3  │ (None, 79, 64)    │          0 │ time_distributed… │
│ (TimeDistributed)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 32)        │          0 │ batch_normalizat… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ (None, 128)       │     98,816 │ time_distributed… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 128)       │      4,224 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 256)       │          0 │ lstm[0][0],       │
│ (Concatenate)       │                   │            │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 128)       │     32,896 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 128)       │          0 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 1)         │        129 │ dropout[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 221,153 (863.88 KB)

 Trainable params: 221,025 (863.38 KB)

 Non-trainable params: 128 (512.00 B)

In [ ]:
fmri_data = np.random.rand(100, 5, 4, 4, 4, 1)
smri_data = np.random.rand(100, 10, 10, 10, 1)
labels = np.random.randint(0, 2, size=(100,))
fmri_data.shape, smri_data.shape, labels.shape

((100, 5, 4, 4, 4, 1), (100, 10, 10, 10, 1), (100,))

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train_fmri, val_fmri, train_smri, val_smri, train_labels, val_labels = train_test_split(fmri_data, smri_data, labels, test_size=0.2, random_state=42)
train_fmri.shape, val_fmri.shape, train_smri.shape, val_smri.shape, train_labels.shape, val_labels.shape

((80, 5, 4, 4, 4, 1),
 (20, 5, 4, 4, 4, 1),
 (80, 10, 10, 10, 1),
 (20, 10, 10, 10, 1),
 (80,),
 (20,))

In [11]:
history = model.fit(
    {"fmri_input": func_train, "smri_input": struct_train},  # Dictionary format for inputs
    y_train,  # Output labels
    batch_size=2,
    epochs=10,
    validation_data=(
        {"fmri_input": func_test, "smri_input": struct_test},
        y_test
    ),
    verbose=1
)

Epoch 1/10


I0000 00:00:1741364874.121657 4136542 service.cc:148] XLA service 0x7e6a38006930 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1741364874.121787 4136542 service.cc:156]   StreamExecutor device (0): Host, Default Version
2025-03-07 21:57:55.969513: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1741364911.921231 4136542 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
2025-03-07 21:58:41.934406: W external/local_xla/xla/tsl/framework/bfc_allocator.cc:497] Allocator (mklcpu) ran out of memory trying to allocate 2.41TiB (rounded to 2653156464640)requested by op 
2025-03-07 21:58:41.934475: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1053] BFCAllocator dump for mklcpu
2025-03-07 21:58:41.934497: I external/local_xla/xla/tsl/framework/bfc

ResourceExhaustedError: Graph execution error:

Detected at node StatefulPartitionedCall defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/usr/local/lib/python3.10/dist-packages/ipykernel_launcher.py", line 18, in <module>

  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 739, in start

  File "/usr/local/lib/python3.10/dist-packages/tornado/platform/asyncio.py", line 205, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelbase.py", line 545, in dispatch_queue

  File "/usr/loc